# Model Preprocessing
---
This notebook performs the necessary preprocessing of CanRCM4 models. Execute each cell in order.

* Place netCDF models into `climpyrical/data/model_inputs`. 
* Ensure ensemble models have:
    * lat, lon, rlat, rlon and a 2D data field variable
    
Output directories are configurable, but are recommended to be placed in `climpyrical/data/results/intermediate/preprocessed_netcdf/`

Workflow:

<img src="../../docs/assets/flowchart_model.png" alt="flowchart" width="450"/>

In [ ]:
from climpyrical.data import read_data, gen_dataset, interpolate_dataset
from climpyrical.gridding import regrid_ensemble, extend_north

from pkg_resources import resource_filename
import warnings

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

### Parameters are loaded from config_example.yml
Example configuration from config_example.yml:
```
    paths:
        preprocessed_model_path: /data/results/intermediate/preprocessed_netcdf/
    RL50:
        station_dv: "RL50 (kPa)"
        station_path: 'data/station_inputs/Interim_snow_rain_load_LR_composite_stations_tbd_v4.csv'
        input_model_path: 'data/model_inputs/snw_rain_CanRCM4-LE_ens35_1951-2016_max_rl50_load_ensmean.nc'
        medians: 
            value: 0.4
            action: "multiply"
        fill_glaciers: True
```

In [ ]:
# Parameters cell

Load the CanRCM4 model at native resolution

In [ ]:
ds = read_data(resource_filename("climpyrical", input_model_path))
(dv,) = ds.data_vars
unit = ds[dv].attrs["units"]
print(f"Design value header name: {dv}")
print(f"Units in model file: {unit}")

After loading, check for accepted units and convert `degC` to `K` if necessary

In [ ]:
rlon, rlat = np.meshgrid(ds.rlon, ds.rlat)
mean = ds[dv].values

accepted_units = ["kPa", "kPas", "Pas" "Pa", "degC", "degC-day", "mm", "unitless", "%"]

print(f"Detect units: {unit}")
if unit not in accepted_units:
    warnings.warn(
        f"{unit} not recognized from list of accepted units: {accepted_units}"
    )

print(f"Units in model file: {unit}")

Load the masks, starting with the land mask at the native CanRCM4 resolution and the glacier mask

In [ ]:
path_mask = resource_filename("climpyrical", "data/masks/land_mask_CanRCM4_sftlf.nc")
path_glacier_mask = resource_filename("climpyrical", "data/masks/glacier_mask.nc")

print("Load and regrid file to target resolution")

mask = read_data(path_mask)
mask = regrid_ensemble(mask, "sftlf", 10, copy=True)
mask = mask["sftlf"] >= 1.0

print("Load original reoslution mask for reference")
mask_og = read_data(path_mask)["sftlf"].values != 0.0
glaciermask = read_data(path_glacier_mask)["mask"].values != 0.0

plot masks as example

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(mask_og, origin="lower", cmap="Greys")
ax[0].set_title("Original mask")
ax[1].imshow(glaciermask, origin="lower", cmap="Greys")
ax[1].set_title("Glacier mask")

Perform bilinear interpolation over glacier points if required

In [ ]:
if fill_glaciers:
    print("Filling spurious glacier points.")
    mean[glaciermask] = np.nan

nanmask = ~np.isnan(mean)
points = np.stack([rlon[nanmask], rlat[nanmask]]).T
target_values = mean[nanmask]
target_points = np.stack([rlon[glaciermask], rlat[glaciermask]]).T

mean[glaciermask] = interpolate_dataset(
    points, target_values, target_points, "linear"
)

ds_filled = gen_dataset(dv, mean, ds.rlat, ds.rlon, ds.lat, ds.lon, unit)


In [ ]:
print("Remove water cells at original resolution")
ds_filled[dv].values[~mask_og] = np.nan
nanmask = ~np.isnan(ds_filled[dv].values)

In [ ]:
print("Copying and downscaling dataset 10x")
ds10 = regrid_ensemble(ds_filled, dv, 10, copy=True)
ds10[dv].values[~mask] = np.nan
nrlon, nrlat = np.meshgrid(ds10.rlon, ds10.rlat)
nanmask10 = ~np.isnan(ds10[dv].values)

In [ ]:
print("Interpolating full remaining grid")
points = np.stack([rlon[nanmask], rlat[nanmask]]).T
target_points = np.stack([nrlon[nanmask10], nrlat[nanmask10]]).T
values = ds[dv].values[nanmask]
ds10[dv].values[nanmask10] = interpolate_dataset(
    points, values, target_points, "linear"
)

In [ ]:
print("Add northern domain to model")
ds10 = extend_north(ds10, dv, 210, fill_val=np.nan)
nanmask10 = ~np.isnan(ds10[dv].values)

Load the higher resolution canada mask

In [ ]:
canada_mask_path = resource_filename("climpyrical", "/data/masks/canada_mask_rp.nc")
with read_data(canada_mask_path) as ds_canada:
    ca_mask = ds_canada["mask"].values

# select NaN values within new mask
ca_mask_or = ~np.logical_or(~ca_mask, nanmask10)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(ca_mask, origin="lower", cmap="Greys")
ax[0].set_title("Target mask")
ax[1].imshow(ca_mask_or, origin="lower", cmap="Greys")
ax[1].set_title("Missing points mask")

In [ ]:
print("Fill remaining missing points using closest neighbour.")
nrlon, nrlat = np.meshgrid(ds10.rlon.values, ds10.rlat.values)

temp_field = ds10[dv].values

points = np.stack([nrlon[nanmask10], nrlat[nanmask10]]).T
target_points = np.stack([nrlon[ca_mask_or], nrlat[ca_mask_or]]).T
target_values = ds10[dv].values[nanmask10]
temp_field[~ca_mask] = np.nan

temp_field[ca_mask_or] = interpolate_dataset(
    points, target_values, target_points, "nearest"
)

In [ ]:
print("Remove the processed northern region.")
uaa_mask_path = resource_filename(
    "climpyrical", "/data/masks/canada_mask_north_rp.nc"
)
uaa_mask = read_data(uaa_mask_path)["mask"]
temp_field[uaa_mask] = np.nan

ds_processed = gen_dataset(
    dv, temp_field, ds10.rlat, ds10.rlon, ds10.lat, ds10.lon, unit
)
# Populate new processed dataset with attributes from 
# the original
if ds.attrs:
    all_keys = set(ds_processed.variables).union(set(ds_processed.dims))
    for key in all_keys:
        ds_processed[key].attrs = ds[key].attrs
    attr_dict = ds.attrs
    attr_dict["Climpyrical"] = (
        "CanRCM4 Reconstruction contains"
        "hybrid station and model data using"
        "Climpyrical (https://github.com/pacificclimate/climpyrical)"
    )

    ds_processed.attrs = attr_dict

print("Dataset generated and writing to file.")

ds_processed.to_netcdf(resource_filename("climpyrical", f"{preprocessed_model_path}{name}.nc"), mode="w")

print("Completed!")

Plot the final result

In [ ]:
plt.figure(figsize=(15, 15))
plt.title(f"Processed {name}")
plt.imshow(ds_processed[dv].values, origin="lower")
plt.colorbar(fraction=0.045)